In [2]:
import sys
sys.path.insert(0,r"C:\Users\pc\Documents")
import numpy as np
import pandas as pd
import csv
from sklearn.model_selection import train_test_split
import class_report as cr
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, auc, f1_score, log_loss
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [3]:
## Lectura de datos     
base_centro = pd.read_csv(r'C:\Users\pc\Downloads\BASE_CENTRO.txt', sep=r'¡', engine='python')
base_centro.head()

## Eliminación de variables: 
base_centro2 = base_centro[[
        #'CUCONUSE', 
        'SESUSERV', 'SERVDESC', 'SESUSUSC', 'SESUFEIN','DEPADESC', 'SESUCUSA', 'SESUSAPE','SESUCICL', 'SUSCNITC', 
        #'VECTOR','SESUCATE', 'SESUSUCA','SESULOCA','SESUSAAN','V1', 'V2', 'V3', 'V4', 'V5', 'V6',#'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 
        'TIPIFICACION_CLIENTE', 'SUMA','CANTIDAD','CALIFICACION_VECTOR_SERVICIO',
       #'VECTOR_CUALITATIVO_CONTRATO',#'BANCO_1', 'DIA_PAGO_1', 'DIA_PAGO_2','DIA_PAGO_3', 'FECHA_SUSPENSION','RANGO_EDAD', 'GENERO',
        'PROM_SUSC', 'CANAL_ENVIO', 'SEGMENTACION', 'REGIONAL','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]

In [4]:
# Vista previa
base_centro2.head()

,SESUSERV,SERVDESC,SESUSUSC,SESUFEIN,DEPADESC,SESUCUSA,SESUSAPE,SESUCICL,SUSCNITC,TIPIFICACION_CLIENTE,...,CANTIDAD,CALIFICACION_VECTOR_SERVICIO,PROM_SUSC,CANAL_ENVIO,SEGMENTACION,REGIONAL,ESTRATO_AGRUPADO,DEPARTAMENTO_AGR,ANTIGUEDAD_DIAS,CANAL_PAGO
0,7239,TELEVISIÓN,17021486,17/12/2019,BOGOTA D.C.,1,56228,7,79848450,NUEVO,...,3,92,"42765,2",DIGITAL,Hogares,CENTRO,2.0,BOGOTA D.C.,154,FISICO
1,7255,TELEFONÍA,17022634,18/12/2019,BOGOTA D.C.,1,28817,7,1014293952,NUEVO,...,4,88,"38009,4",DIGITAL,Hogares,CENTRO,3.0,BOGOTA D.C.,153,FISICO
2,7245,INTERNET,17070737,14/01/2020,BOGOTA D.C.,1,40000,7,4096838,NUEVO,...,2,83,25781,DIGITAL,Hogares,CENTRO,4.0,BOGOTA D.C.,126,FISICO
3,7255,TELEFONÍA,17060437,09/01/2020,BOGOTA D.C.,1,30261,7,1070328828,NUEVO,...,3,83,"38662,5",DIGITAL,Hogares,CENTRO,2.0,BOGOTA D.C.,131,DIGITAL
4,7245,INTERNET,17060514,09/01/2020,BOGOTA D.C.,1,41408,7,1088310422,NUEVO,...,2,88,"23981,67",DIGITAL,Hogares,CENTRO,2.0,BOGOTA D.C.,131,DIGITAL


In [5]:
## Numero completo de clientes en la region
len(base_centro2)

482220

In [6]:
## Eliminacion de clientes con datos NA
base_centro3 = base_centro2.dropna()

In [7]:
len(base_centro3)

441239

In [8]:
## Group by por nit para calificacion
b4 = base_centro3[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO']]
group_nit = b4.groupby(['SUSCNITC'])['CALIFICACION_VECTOR_SERVICIO'].mean().reset_index()


## Join de la nueva calificacion
base_centro4 = pd.merge(base_centro3, group_nit, how='left', on=['SUSCNITC'])

## Prueba promedio
b5 = base_centro4[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO_x','CALIFICACION_VECTOR_SERVICIO_y']]

### Creacion de la etiqueta de clasificacion
base_centro4['y'] = pd.cut(x=base_centro4['CALIFICACION_VECTOR_SERVICIO_y'],
                            bins=[-1,50,76,100],
                            labels=['No pago','Pago inoportuno','Pago'])


In [9]:
dataTypeSeries = base_centro4.dtypes
dataTypeSeries

cat = base_centro4.select_dtypes(include=['object','category'])
cuanti = base_centro4.select_dtypes(include=['int64','float64'])
  
cat = cat.apply(lambda col: LabelEncoder().fit_transform(col))

In [10]:
cuanti.corr(method = 'pearson')

,SESUSERV,SESUSUSC,SESUCUSA,SESUCICL,SUMA,CANTIDAD,CALIFICACION_VECTOR_SERVICIO_x,ESTRATO_AGRUPADO,ANTIGUEDAD_DIAS,CALIFICACION_VECTOR_SERVICIO_y
SESUSERV,1.000000,0.359883,0.040173,-0.098872,-0.154939,-0.117563,-0.126885,0.365878,-0.381001,-0.093071
SESUSUSC,0.359883,1.000000,0.006084,-0.241694,-0.288587,-0.304650,-0.148147,0.135241,-0.087827,-0.142206
SESUCUSA,0.040173,0.006084,1.000000,0.022404,-0.310200,-0.013828,-0.406735,-0.073887,-0.012140,-0.404401
SESUCICL,-0.098872,-0.241694,0.022404,1.000000,0.459425,0.493918,0.219458,-0.272025,0.269889,0.226101
SUMA,-0.154939,-0.288587,-0.310200,0.459425,1.000000,0.897093,0.726616,-0.025014,0.014697,0.703844
CANTIDAD,-0.117563,-0.304650,-0.013828,0.493918,0.897093,1.000000,0.511450,-0.040679,0.022635,0.485971
CALIFICACION_VECTOR_SERVICIO_x,-0.126885,-0.148147,-0.406735,0.219458,0.726616,0.511450,1.000000,0.001540,0.028383,0.956594
ESTRATO_AGRUPADO,0.365878,0.135241,-0.073887,-0.272025,-0.025014,-0.040679,0.001540,1.000000,-0.608978,0.001610
ANTIGUEDAD_DIAS,-0.381001,-0.087827,-0.012140,0.269889,0.014697,0.022635,0.028383,-0.608978,1.000000,0.029671
CALIFICACION_VECTOR_SERVICIO_y,-0.093071,-0.142206,-0.404401,0.226101,0.703844,0.485971,0.956594,0.001610,0.029671,1.000000


In [11]:
base_centro5=cuanti.join(cat)
print(base_centro5.dtypes)

base_centro5.to_csv(r'C:\Users\pc\Desktop\base_centro_process.csv')

#base_centro5 = pd.read_csv(r'C:\Users\pc\Desktop\base_centro_process.csv', sep=',', engine='python')

X = base_centro5[[
        #'CUCONUSE', 
        'SESUSERV', 'SERVDESC', 
        #'SESUSUSC',
        'DEPADESC', 'SESUCUSA', 'SESUSAPE','SESUCICL', 'TIPIFICACION_CLIENTE', 
        #'SESUFEIN','SUSCNITC', 'VECTOR','SESUCATE', 'SESUSUCA','SESULOCA','SESUSAAN','RANGO_EDAD', 'GENERO', 
        #'V1', 'V2', 'V3', 'V4', 'V5', 'V6','V7', 'V8', 'V9', 'V10', 'V11', 'V12',
        #'REGIONAL',
        'PROM_SUSC', 'CANAL_ENVIO',
        # 'SUMA',
        #'CANTIDAD',
    #'CALIFICACION_VECTOR_SERVICIO_x','CALIFICACION_VECTOR_SERVICIO_y','DEPARTAMENTO_AGR'
        #'VECTOR_CUALITATIVO_CONTRATO','BANCO_1', 'DIA_PAGO_1', 'DIA_PAGO_2','DIA_PAGO_3', 'FECHA_SUSPENSION', 
       'SEGMENTACION', 'ESTRATO_AGRUPADO','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]

y = base_centro5['y']

SESUSERV                            int64
SESUSUSC                            int64
SESUCUSA                            int64
SESUCICL                            int64
SUMA                                int64
CANTIDAD                            int64
CALIFICACION_VECTOR_SERVICIO_x      int64
ESTRATO_AGRUPADO                  float64
ANTIGUEDAD_DIAS                     int64
CALIFICACION_VECTOR_SERVICIO_y    float64
SERVDESC                            int32
SESUFEIN                            int32
DEPADESC                            int32
SESUSAPE                            int32
SUSCNITC                            int32
TIPIFICACION_CLIENTE                int32
PROM_SUSC                           int32
CANAL_ENVIO                         int32
SEGMENTACION                        int32
REGIONAL                            int32
DEPARTAMENTO_AGR                    int32
CANAL_PAGO                          int32
y                                   int32
dtype: object


In [12]:
## Vista previa de las variables preliminares para clasificacion
X.head()

,SESUSERV,SERVDESC,DEPADESC,SESUCUSA,SESUSAPE,SESUCICL,TIPIFICACION_CLIENTE,PROM_SUSC,CANAL_ENVIO,SEGMENTACION,ESTRATO_AGRUPADO,ANTIGUEDAD_DIAS,CANAL_PAGO
0,7239,32,0,1,44576,7,1,94010,0,1,2.0,154,1
1,7255,29,0,1,30119,7,1,70370,0,1,3.0,153,1
2,7245,9,0,1,35953,7,1,14062,0,1,4.0,126,1
3,7255,29,0,1,30892,7,1,74554,0,1,2.0,131,0
4,7245,9,0,1,36696,7,1,10616,0,1,2.0,131,0


In [13]:
from datetime import datetime
instanteInicial = datetime.now()
X_ = StandardScaler().fit_transform(X)
#X_ = X
X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2,random_state=321)

classifiers = [
    KNeighborsClassifier(3),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=500),
    LogisticRegression(multi_class="multinomial", solver="lbfgs", C=5),
    GaussianNB()
    ]

# Competencia de clasificadores
log_cols=["Classifier", "Accuracy", "Log Loss", "F1_SCORE"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__  
    print("="*30)
    print(name)
    print('****Resultados****')    
    train_predictions = clf.predict_proba(X_test)
    acc = cr.class_report(y_true = y_test, 
                                y_pred = clf.predict(X_test),
                                y_score = train_predictions)
    print("Métricas: {}".format(acc))
    log_entry = pd.DataFrame([[name, acc]])
    log = log.append(log_entry)
    
print("="*30)

C:\Users\pc\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\pc\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.850486  0.841013  0.845723  13825.0  13671.0  0.949065
1             0.859811  0.892891  0.876039  46915.0  48720.0  0.910081
2             0.765015  0.719100  0.741347  27508.0  25857.0  0.866884
avg / total   0.828801  0.830591  0.829305  88248.0  88248.0  0.922770
DecisionTreeClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.962217  0.753418  0.845112  13825.0  10825.0  0.950477
1             0.820993  0.919621  0.867513  46915.0  52551.0  0.905471
2             0.760775  0.687873  0.722489  27508.0  24872.0  0.873377
avg / total   0.824347  0.821344  0.818798  88248.0  88248.0  0.938133
RandomForestClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.945385  0.606004  0.738573  13825.0   8862.0  0.9

In [14]:
# Importancia de variables
model = DecisionTreeClassifier(max_depth=5).fit(X_train, y_train)

importance = model.feature_importances_
# Total importancia de variables
for i,v in enumerate(importance):
	print('Variable: %0d, Score: %.5f' % (i,v))

Variable: 0, Score: 0.00000
Variable: 1, Score: 0.00189
Variable: 2, Score: 0.00000
Variable: 3, Score: 0.39017
Variable: 4, Score: 0.06967
Variable: 5, Score: 0.04650
Variable: 6, Score: 0.00000
Variable: 7, Score: 0.02148
Variable: 8, Score: 0.00000
Variable: 9, Score: 0.01825
Variable: 10, Score: 0.00004
Variable: 11, Score: 0.30926
Variable: 12, Score: 0.14275


In [ ]:
## El mejor modelo de clasificacion fue el KNN, veamos el k optimo con corridas de k = 2 a k = 10

In [15]:
classifiers = [
    KNeighborsClassifier(2),
    KNeighborsClassifier(3),
    KNeighborsClassifier(4),
    KNeighborsClassifier(5),
    KNeighborsClassifier(6),
    KNeighborsClassifier(7),
    KNeighborsClassifier(8),
    KNeighborsClassifier(9),
    KNeighborsClassifier(10)
    ]

# Competencia de clasificadores
log_cols=["Classifier", "Accuracy", "Log Loss", "F1_SCORE"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__  
    print("="*30)
    print(name)
    print('****Resultados****')    
    train_predictions = clf.predict_proba(X_test)
    acc = cr.class_report(y_true = y_test, 
                                y_pred = clf.predict(X_test),
                                y_score = train_predictions)
    print("Métricas: {}".format(acc))
    log_entry = pd.DataFrame([[name, acc]])
    log = log.append(log_entry)
    
print("="*30)

KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.786337  0.894177  0.836797  13825.0  15721.0  0.937289
1             0.818373  0.929106  0.870231  46915.0  53263.0  0.893315
2             0.827606  0.579577  0.681733  27508.0  19264.0  0.845309
avg / total   0.816232  0.814681  0.806236  88248.0  88248.0  0.905763
KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.850486  0.841013  0.845723  13825.0  13671.0  0.949065
1             0.859811  0.892891  0.876039  46915.0  48720.0  0.910081
2             0.765015  0.719100  0.741347  27508.0  25857.0  0.866884
avg / total   0.828801  0.830591  0.829305  88248.0  88248.0  0.922770
KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.852960  0.861844  0.857379  13825.0  13969.0  0.95596

In [16]:
kf = StratifiedKFold(n_splits=5,shuffle=False)

accuracy_model = []
# Iteraciones del entrenamiento sobre los kfolds
X_fin = X[['SESUSERV','SESUCUSA', 'SESUSAPE','SESUCICL', 'TIPIFICACION_CLIENTE', 'PROM_SUSC',
'CANAL_ENVIO','SEGMENTACION', 'ESTRATO_AGRUPADO','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]
X_ = StandardScaler().fit_transform(X_fin)

for train_index, test_index in kf.split(X_, y) :
    # Particion train test para validacion cruzada
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Entrenamiento de la red MLP
    model = KNeighborsClassifier(7).fit(X_train, y_train) 
    accuracy_model.append(cr.class_report(y_true = y_test, 
                                       y_pred = model.predict(X_test),
                                      y_score = model.predict_proba(X_test)))
 
# Print the accuracy    
print(accuracy_model)

C:\Users\pc\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\pc\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


[             precision    recall  f1-score  support     pred       AUC
0             0.766596  0.708491  0.736399  13708.0  12669.0  0.910172
1             0.777613  0.839710  0.807469  46784.0  50520.0  0.857817
2             0.643336  0.580826  0.610485  27757.0  25060.0  0.797786
avg / total   0.733667  0.737901  0.734472  88249.0  88249.0  0.883018,              precision    recall  f1-score  support     pred       AUC
0             0.726112  0.721768  0.723934  13708.0  13626.0  0.910311
1             0.766394  0.854135  0.807889  46783.0  52139.0  0.860335
2             0.639194  0.517743  0.572094  27757.0  22483.0  0.788435
avg / total   0.720128  0.727767  0.720682  88248.0  88248.0  0.879877,              precision    recall  f1-score  support     pred       AUC
0             0.751542  0.702145  0.726004  13708.0  12807.0  0.905605
1             0.783494  0.844773  0.812980  46783.0  50442.0  0.871148
2             0.654906  0.589833  0.620669  27757.0  24999.0  0.820432
avg

In [ ]:
## Matriz de Confuncion

In [17]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, model.predict(X_test))

array([[ 9008,  1686,  3013],
       [  976, 39979,  5828],
       [ 2847, 11049, 13861]], dtype=int64)